In [1]:
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from matplotlib import pyplot as plt
from PIL import Image
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
veri = []
etiketler = []
siniflar = 43
klasorler = os.getcwd() # Bu fonksiyon sayesinde içinde bulunduğumuz dizini değiştirebiliriz.

In [3]:
#Resimleri ve etiketlerini alma
for i in range(siniflar):
    klasor = os.path.join(klasorler,'Egitim',str(i))
    img = os.listdir(klasor)

    for a in img:
        try:
            img = Image.open(klasor + '\\'+ a)
            img = img.resize((30,30))
            img = np.array(img)
            veri.append(img)
            etiketler.append(i)
        except:
            print("Görsel Düzgün Yüklenemedi!!!")

In [4]:
#Listeleri numpy dizilerine dönüştürme
veri = np.array(veri)
etiketler = np.array(etiketler)

In [7]:
print(veri.shape, etiketler.shape)

(72238, 30, 30, 3) (72238,)


In [8]:
#Eğitim veri test veri kümesini bölme
X_train, X_test, y_train, y_test = train_test_split(veri, etiketler, test_size=0.2, random_state=42)


#Etiketleri tek bir etkin kodlamaya dönüştürme
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)


In [9]:
#Modeli oluşturma
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Modeli tamamlama
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("model.h5")

Epoch 1/15
1806/1806 [==============================] - 40s 22ms/step - loss: 1.5140 - accuracy: 0.5898 - val_loss: 0.3459 - val_accuracy: 0.9065
Epoch 2/15
1806/1806 [==============================] - 40s 22ms/step - loss: 0.6653 - accuracy: 0.8008 - val_loss: 0.2608 - val_accuracy: 0.9362
Epoch 3/15
1295/1806 [====================>.........] - ETA: 10s - loss: 0.5036 - accuracy: 0.8451

KeyboardInterrupt: 

In [ ]:
#doğruluk için grafik çizme
plt.figure(0)
plt.plot(history.history['accuracy'], label='eğitim doğruluğu')
plt.plot(history.history['val_accuracy'], label='değer doğruluğu')
plt.title('Doğruluk')
plt.xlabel('eğitim tur sayısı')
plt.ylabel('doğruluk')
plt.legend()
plt.show()

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'], label='eğitim kaybı')
plt.plot(history.history['val_loss'], label='değer kaybı')
plt.title('Kayıp')
plt.xlabel('eğitim tur sayısı')
plt.ylabel('Kayıp')
plt.legend()
plt.show()

In [ ]:
#test veri setinde test doğruluğu
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')

etiketler = y_test["ClassId"].values
imgs = y_test["Path"].values

veri=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    veri.append(np.array(image))

X_test=np.array(veri)
tahmin = model.predict(X_test).argmax(axis=1)

In [ ]:
#Test verileriyle doğruluk
from sklearn.metrics import accuracy_score
print(accuracy_score(etiketler, tahmin))

In [ ]:
model.save('model.h5')